<a href="https://colab.research.google.com/github/liuxb555/SkillMetrics/blob/master/Google_Colab_SigleModel_FFC_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# If not on Colab you'll need install the earth-engine Python API
# !pip install earthengine-api #earth-engine Python API
# Earth Engine Python API
# Athenticate to your GEE account. 
!earthengine authenticate
import ee 
ee.Initialize()

# Connect Google Drive  注意，每次打开新的Colab Notebook都需要重新关联。
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


#install python packages to run this notebook 
!pip install -q rasterio

import numpy as np
import pandas as pd
import os
import pickle
import rasterio
import glob
import warnings
from os import path as op
warnings.filterwarnings('ignore')  # 忽略warnings

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=sp4e2i4FIWTcsu9BzhYM__LnNP8hdwZzrGdwgAufk7I&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgeQ_lTt6NBfSh7OuTgCdRI0lbW_6-vdIwbmPNWDJof8TQ8NPjJPww

Successfully saved authorization token.
Mounted at /content/drive/
     |████████████████████████████████| 19.3 MB 161 kB/s 

通过波段转换，L457的波段转换为OLI(Landsat-8)的波段

In [2]:
%%time

#install python packages to run this notebook 
!pip install -q rasterio

import numpy as np
import pandas as pd
import os
import pickle
import rasterio
import glob
import warnings
from os import path as op
warnings.filterwarnings('ignore')  # 忽略warnings

# 通过波段转换，L457的波段转换为OLI(Landsat-8)的波段
#  landsat5(TM)
def TMtoOLI(TM1, TM2, TM3, TM4, TM5, TM7): 
    # OLI(Band1)	1.3063,	-0.3344	-0.015	0.0443	-0.0516	0.0453	-37
    # OLI(Band2)	0.9704	0.0303	0.0042	-0.005	0.0058	-0.0053	4
    # OLI(Band3)	-0.051	1.095	-0.0481	0.0039	0.0002	-0.0001	-3
    # OLI(Band4)	-0.0039	0.0437	0.9634	-0.0018	-0.0086	0.0055	3
    # OLI(Band5)	0.0666	-0.157	0.0642	1.0079	0.0386	-0.0225	-1
    # OLI(Band6)	-0.0354	0.0485	0.0042	-0.0405	1.0941	-0.0929	29
    # OLI(Band7)	-0.0004	0.0497	-0.0398	0.0014	0.0497	0.9607	-25
    TM = [TM1/0.0001, TM2/0.0001, TM3/0.0001, TM4/0.0001, TM5/0.0001, TM7/0.0001]
    BCC1 = [1.3509,-0.4224,0.0219,0.0516,-0.071,0.0598]
    BCC2 = [1.0429,-0.0606,0.16,0.0021,-0.0042,0.0027]
    BCC3 = [0.0087,1.1395,-0.1538,0.0045,0.0102,-0.0065]
    BCC4 = [0.0166,0.0041,0.9876,-0.007,-0.008,0.0066]
    BCC5 = [0.0599,-0.1590,0.0758,1.0088,0.0314,-0.0193]
    BCC6 = [-0.0555,0.0768,-0.0061,-0.0492,1.1231,-0.1244]
    BCC7 = [-0.0054,0.0594,-0.0368,-0.0128,0.0702,0.9438]


    OLI_1 = sum(list(map(lambda x, y: x * y, TM, BCC1))) - 45
    OLI_2 = sum(list(map(lambda x, y: x * y, TM, BCC2))) - 1
    OLI_3 = sum(list(map(lambda x, y: x * y, TM, BCC3))) - 5
    OLI_4 = sum(list(map(lambda x, y: x * y, TM, BCC4))) + 3
    OLI_5 = sum(list(map(lambda x, y: x * y, TM, BCC5))) - 1
    OLI_6 = sum(list(map(lambda x, y: x * y, TM, BCC6))) + 1
    OLI_7 = sum(list(map(lambda x, y: x * y, TM, BCC7))) - 16

    OLI_1 = OLI_1*0.0001
    OLI_2 = OLI_2*0.0001
    OLI_3 = OLI_3*0.0001
    OLI_4 = OLI_4*0.0001
    OLI_5 = OLI_5*0.0001
    OLI_6 = OLI_6*0.0001
    OLI_7 = OLI_7*0.0001

    return OLI_1,OLI_2,OLI_3,OLI_4,OLI_5,OLI_6,OLI_7

# Landsat-7 Enhanced Thematic Mapper ，ETM+
def ETMtoOLI(ETM1, ETM2, ETM3, ETM4, ETM5, ETM7):
    # OLI(Band1)	1.3063,	-0.3344	-0.015	0.0443	-0.0516	0.0453	-37
    # OLI(Band2)	0.9704	0.0303	0.0042	-0.005	0.0058	-0.0053	4
    # OLI(Band3)	-0.051	1.095	-0.0481	0.0039	0.0002	-0.0001	-3
    # OLI(Band4)	-0.0039	0.0437	0.9634	-0.0018	-0.0086	0.0055	3
    # OLI(Band5)	0.0666	-0.157	0.0642	1.0079	0.0386	-0.0225	-1
    # OLI(Band6)	-0.0354	0.0485	0.0042	-0.0405	1.0941	-0.0929	29
    # OLI(Band7)	-0.0004	0.0497	-0.0398	0.0014	0.0497	0.9607	-25
    ETM = [ETM1/0.0001, ETM2/0.0001, ETM3/0.0001, ETM4/0.0001, ETM5/0.0001, ETM7/0.0001]
    BCC1=[1.3063,-0.3344,-0.015,0.0443,-0.0516,0.0453]
    BCC2=[0.9704,0.0303,0.0042,-0.005,0.0058,-0.0053]
    BCC3=[-0.051,1.095,-0.0481,0.0039,0.0002,-0.0001]
    BCC4=[-0.0039,0.0437,0.9634,-0.0018,-0.0086,0.0055]
    BCC5=[0.0666,-0.157,0.0642,1.0079,0.0386,-0.0225]
    BCC6=[-0.0354,0.0485,0.0042,-0.0405,1.0941,-0.0929]
    BCC7=[-0.0004,0.0497,-0.0398,0.0014,0.0497,0.9607]

    OLI_1=sum(list(map(lambda x,y:x*y,ETM,BCC1))) -37
    OLI_2=sum(list(map(lambda x,y:x*y,ETM,BCC2))) +4
    OLI_3=sum(list(map(lambda x,y:x*y,ETM,BCC3))) -3
    OLI_4=sum(list(map(lambda x,y:x*y,ETM,BCC4))) +3
    OLI_5=sum(list(map(lambda x,y:x*y,ETM,BCC5))) -1
    OLI_6=sum(list(map(lambda x,y:x*y,ETM,BCC6))) +29
    OLI_7=sum(list(map(lambda x,y:x*y,ETM,BCC7))) -25

    OLI_1 = OLI_1*0.0001
    OLI_2 = OLI_2*0.0001
    OLI_3 = OLI_3*0.0001
    OLI_4 = OLI_4*0.0001
    OLI_5 = OLI_5*0.0001
    OLI_6 = OLI_6*0.0001
    OLI_7 = OLI_7*0.0001

    return OLI_1,OLI_2,OLI_3,OLI_4,OLI_5,OLI_6,OLI_7

CPU times: user 12.2 ms, sys: 16.5 ms, total: 28.7 ms
Wall time: 2.43 s


In [ ]:
%%time

%%time

#install python packages to run this notebook 
!pip install -q rasterio

import numpy as np
import pandas as pd
import os
import pickle
import rasterio
import glob
import warnings
from os import path as op
warnings.filterwarnings('ignore')  # 忽略warnings

# 通过波段转换，L457的波段转换为OLI(Landsat-8)的波段
#  landsat5(TM)
def TMtoOLI(TM1, TM2, TM3, TM4, TM5, TM7): 
    # OLI(Band1)	1.3063,	-0.3344	-0.015	0.0443	-0.0516	0.0453	-37
    # OLI(Band2)	0.9704	0.0303	0.0042	-0.005	0.0058	-0.0053	4
    # OLI(Band3)	-0.051	1.095	-0.0481	0.0039	0.0002	-0.0001	-3
    # OLI(Band4)	-0.0039	0.0437	0.9634	-0.0018	-0.0086	0.0055	3
    # OLI(Band5)	0.0666	-0.157	0.0642	1.0079	0.0386	-0.0225	-1
    # OLI(Band6)	-0.0354	0.0485	0.0042	-0.0405	1.0941	-0.0929	29
    # OLI(Band7)	-0.0004	0.0497	-0.0398	0.0014	0.0497	0.9607	-25
    TM = [TM1/0.0001, TM2/0.0001, TM3/0.0001, TM4/0.0001, TM5/0.0001, TM7/0.0001]
    BCC1 = [1.3509,-0.4224,0.0219,0.0516,-0.071,0.0598]
    BCC2 = [1.0429,-0.0606,0.16,0.0021,-0.0042,0.0027]
    BCC3 = [0.0087,1.1395,-0.1538,0.0045,0.0102,-0.0065]
    BCC4 = [0.0166,0.0041,0.9876,-0.007,-0.008,0.0066]
    BCC5 = [0.0599,-0.1590,0.0758,1.0088,0.0314,-0.0193]
    BCC6 = [-0.0555,0.0768,-0.0061,-0.0492,1.1231,-0.1244]
    BCC7 = [-0.0054,0.0594,-0.0368,-0.0128,0.0702,0.9438]


    OLI_1 = sum(list(map(lambda x, y: x * y, TM, BCC1))) - 45
    OLI_2 = sum(list(map(lambda x, y: x * y, TM, BCC2))) - 1
    OLI_3 = sum(list(map(lambda x, y: x * y, TM, BCC3))) - 5
    OLI_4 = sum(list(map(lambda x, y: x * y, TM, BCC4))) + 3
    OLI_5 = sum(list(map(lambda x, y: x * y, TM, BCC5))) - 1
    OLI_6 = sum(list(map(lambda x, y: x * y, TM, BCC6))) + 1
    OLI_7 = sum(list(map(lambda x, y: x * y, TM, BCC7))) - 16

    OLI_1 = OLI_1*0.0001
    OLI_2 = OLI_2*0.0001
    OLI_3 = OLI_3*0.0001
    OLI_4 = OLI_4*0.0001
    OLI_5 = OLI_5*0.0001
    OLI_6 = OLI_6*0.0001
    OLI_7 = OLI_7*0.0001

    return OLI_1,OLI_2,OLI_3,OLI_4,OLI_5,OLI_6,OLI_7

# Landsat-7 Enhanced Thematic Mapper ，ETM+
def ETMtoOLI(ETM1, ETM2, ETM3, ETM4, ETM5, ETM7):
    # OLI(Band1)	1.3063,	-0.3344	-0.015	0.0443	-0.0516	0.0453	-37
    # OLI(Band2)	0.9704	0.0303	0.0042	-0.005	0.0058	-0.0053	4
    # OLI(Band3)	-0.051	1.095	-0.0481	0.0039	0.0002	-0.0001	-3
    # OLI(Band4)	-0.0039	0.0437	0.9634	-0.0018	-0.0086	0.0055	3
    # OLI(Band5)	0.0666	-0.157	0.0642	1.0079	0.0386	-0.0225	-1
    # OLI(Band6)	-0.0354	0.0485	0.0042	-0.0405	1.0941	-0.0929	29
    # OLI(Band7)	-0.0004	0.0497	-0.0398	0.0014	0.0497	0.9607	-25
    ETM = [ETM1/0.0001, ETM2/0.0001, ETM3/0.0001, ETM4/0.0001, ETM5/0.0001, ETM7/0.0001]
    BCC1=[1.3063,-0.3344,-0.015,0.0443,-0.0516,0.0453]
    BCC2=[0.9704,0.0303,0.0042,-0.005,0.0058,-0.0053]
    BCC3=[-0.051,1.095,-0.0481,0.0039,0.0002,-0.0001]
    BCC4=[-0.0039,0.0437,0.9634,-0.0018,-0.0086,0.0055]
    BCC5=[0.0666,-0.157,0.0642,1.0079,0.0386,-0.0225]
    BCC6=[-0.0354,0.0485,0.0042,-0.0405,1.0941,-0.0929]
    BCC7=[-0.0004,0.0497,-0.0398,0.0014,0.0497,0.9607]

    OLI_1=sum(list(map(lambda x,y:x*y,ETM,BCC1))) -37
    OLI_2=sum(list(map(lambda x,y:x*y,ETM,BCC2))) +4
    OLI_3=sum(list(map(lambda x,y:x*y,ETM,BCC3))) -3
    OLI_4=sum(list(map(lambda x,y:x*y,ETM,BCC4))) +3
    OLI_5=sum(list(map(lambda x,y:x*y,ETM,BCC5))) -1
    OLI_6=sum(list(map(lambda x,y:x*y,ETM,BCC6))) +29
    OLI_7=sum(list(map(lambda x,y:x*y,ETM,BCC7))) -25

    OLI_1 = OLI_1*0.0001
    OLI_2 = OLI_2*0.0001
    OLI_3 = OLI_3*0.0001
    OLI_4 = OLI_4*0.0001
    OLI_5 = OLI_5*0.0001
    OLI_6 = OLI_6*0.0001
    OLI_7 = OLI_7*0.0001

    return OLI_1,OLI_2,OLI_3,OLI_4,OLI_5,OLI_6,OLI_7

if __name__ == '__main__':

    modelPath = ('/content/drive/MyDrive/GEE_FFC/FFC_EXE')
    # modelR_file = 'GEE_PY_0.55_Merge_Train_40Vis_Merge_Pre_40Vis_62240Vis_RFR_RD_model.sav'   
    # modelR_file = 'GEE_Merge_Train_40Vis-0Samp02_40VIS_005Class_2_Merge_Pre_40Vis_005Class_2_62240Vis_RFR_RD_model.sav'
    modelR_file = 'GEE_Colab_DC2_MER_TRA_40Vis-0Samp02_005Class2_DC2_MER_Pre_40Vis_005Class2_XGBR1.00_N100_Paras003-02_model.sav'
    loaded_modelR = pickle.load(open(op.join(modelPath, modelR_file), 'rb'))
    print(modelR_file)

    # 文件所在路径
    input_path = ('/content/drive/Shareddrives/liuxb555@gmail.com/Landsat_05151015/Landsat_Median_TNR/L8_202005151015_Median_TNR')
    # 文件保存路径
    output_path = ('/content/drive/Shareddrives/liuxb555@gmail.com/Landsat_05151015/Landsat_Median_TNR/L8_202005151015_Median_TNR_FFC')
    if not os.path.exists(output_path):  # 创建目录,若目标目录或文件夹不存在，则自动创建，
        os.makedirs(output_path)
    print('Save Path：--------------->', output_path)

    tiffList = glob.glob(op.join(input_path,'L8_*.tif'))    #  L8_max_2018_EXE_0515-1001-0000000000-0000010496.tif    EXE_iMAGE_max-0000000000-0000000000  L8_median_2018-0000073472-0000220416.tif
    print('len(tiffList) ',len(tiffList))

    for num_i in range (82,len(tiffList)):
        print('(len(tiffList))',(len(tiffList)),'num_i',num_i)
        # raster_file = (os.path.basename(tiffList[num_i]))   # 获取tiff文件名称
        raster_file = ((tiffList[num_i]))           # 获取当前tiff文件名称
        # print('raster_file: ',raster_file)

        (path, filename) = os.path.split(raster_file)  # 拆分路径和文件名成
        (name, extension) = os.path.splitext(filename)  # 拆分文件的扩展名
        # print('path',path)         # 路径
        # print('filename',filename)     # 文件名
        # print('name',name)         # 文件名称
        # print('extension',extension)

        new_image = raster_file
        output_image = op.join(output_path, 'Pred_' + name + '_GEE_Colab_DC2-DC2_XGBR1.00_N100_Paras003-02_FFC.tif')
        print('output_image: ',output_image)


        with rasterio.open(new_image, 'r') as src:
            profile = src.profile
            profile.update(dtype=rasterio.uint8,count=1)
            # profile.update(dtype=rasterio.float32,count=1)

            with rasterio.open(output_image, 'w', **profile) as dst: 

                # perform prediction on each small image patch to minimize required memory
                patch_size = 2500

                # print('src.shape',src.shape)
                # print('((src.shape[0] // patch_size) + 1)',((src.shape[0] // patch_size) + 1))
                # print('((src.shape[1] // patch_size) + 1)',((src.shape[1] // patch_size) + 1))

                for i in range((src.shape[0] // patch_size) + 1):
                    for j in range((src.shape[1] // patch_size) + 1):
                        # print('i', i, 'j', j)
                        # define the pixels to read (and write)
                        window = rasterio.windows.Window(
                            j * patch_size,
                            i * patch_size,
                            # don't read past the image bounds
                            min(patch_size, src.shape[1] - j * patch_size),
                            min(patch_size, src.shape[0] - i * patch_size)
                        )
                        # print('window',window)

                        data = src.read(window=window)
                        # print('data.shape',data.shape)
                        # read the image into the proper format, adding indices if necessary
                        img_swp = np.moveaxis(data, 0, 2)
                        # print('img_swp.shape',img_swp.shape)
                        img_flat = img_swp.reshape(-1, img_swp.shape[-1])
                        # print('img_flat.shape',img_flat.shape)

                        # # Calculate L5
                        # TM1 = img_flat[:,0]   
                        # TM2 = img_flat[:,1]
                        # TM3 = img_flat[:,2]
                        # TM4 = img_flat[:,3]
                        # TM5 = img_flat[:,4]
                        # TM7 = img_flat[:,5]
                        # Aspect = img_flat[:,6]
                        # DEM = img_flat[:, 7]
                        # Lon = img_flat[:, 8]
                        # Lat = img_flat[:, 9]
                        # Corstal, Blue, Green, Red, NIR, SWIR_1, SWIR_2 = TMtoOLI(TM1, TM2, TM3, TM4, TM5, TM7)  # 函数使用的是10000的数据

                        # # Calculate L7
                        # ETM1 = img_flat[:,0]   
                        # ETM2 = img_flat[:,1]
                        # ETM3 = img_flat[:,2]
                        # ETM4 = img_flat[:,3]
                        # ETM5 = img_flat[:,4]
                        # ETM7 = img_flat[:,5]
                        # Aspect = img_flat[:,6]
                        # DEM = img_flat[:, 7]
                        # Lon = img_flat[:, 8]
                        # Lat = img_flat[:, 9]
                        # Corstal, Blue, Green, Red, NIR, SWIR_1, SWIR_2 = ETMtoOLI(ETM1, ETM2, ETM3, ETM4, ETM5, ETM7)  # 函数使用的是10000的数据

                        # Calculate L8
                        Blue = img_flat[:, 0]
                        Green = img_flat[:, 1]
                        Red = img_flat[:, 2]
                        NIR = img_flat[:, 3]
                        SWIR_1 = img_flat[:, 4]
                        SWIR_2 = img_flat[:, 5]
                        Aspect = img_flat[:, 6]
                        DEM = img_flat[:, 7]
                        Lon = img_flat[:, 8]
                        Lat = img_flat[:, 9]

                        img_AFVI1600 = np.expand_dims((NIR - 0.66 * (SWIR_1 / (NIR + 0.66 * SWIR_1))), axis=1)
                        img_ARVI = np.expand_dims(((NIR - (2 * Red - Blue)) / (NIR + (2 * Red - Blue))), axis=1)
                        img_ATSAVI = np.expand_dims((1.22 * ((NIR - 1.22 * Red - 0.33) / (1.22 * NIR + Red + 0.235672))), axis=1)
                        img_BI_1 = np.expand_dims((((SWIR_1 + Red) - (Blue + NIR)) / ((SWIR_1 + Red) + (Blue + NIR))), axis=1)
                        img_BI_2 = np.expand_dims((((SWIR_2 + Red) - (Blue + NIR)) / ((SWIR_2 + Red) + (Blue + NIR))), axis=1)
                        img_CSI = np.expand_dims((NIR / SWIR_1), axis=1)
                        img_DVI = np.expand_dims((NIR - Red), axis=1)
                        img_EVI = np.expand_dims((2.5 * ((NIR - Red) / (NIR + 6 * Red - 7.5 * Blue + 1))), axis=1)
                        img_GARI_0 = np.expand_dims(((NIR - (Green - (Blue - Red))) / (NIR + (Green - (Blue - Red)))), axis=1)
                        img_GB_NDVI = np.expand_dims(((NIR - (Green + Blue)) / (NIR + (Green + Blue))), axis=1)
                        img_GSAVI = np.expand_dims((1.5 * ((NIR - Green) / (NIR + Green + 0.5))), axis=1)
                        img_IPVI = np.expand_dims((NIR / (NIR + Red)), axis=1)
                        img_Laterite = np.expand_dims((SWIR_1 / SWIR_2), axis=1)
                        img_MIRBI = np.expand_dims((10 * SWIR_2 + 9.8 * SWIR_1 + 2), axis=1)
                        img_MNDWI = np.expand_dims(((Green - SWIR_1) / (Green + SWIR_1)), axis=1)
                        img_MSAVI = np.expand_dims((((2 * NIR + 1 - (((2 * NIR + 1) ** 2 - 8 * (NIR - Red)) ** 0.5))) / 2), axis=1)
                        img_MTVI_2 = np.expand_dims(((1.5 * (1.2 * (NIR - Red) - 2.5 * (Red - Green))) / (((2 * NIR + 1) ** 2 - (6 * NIR - 5 * (Red) ** 0.5) - 0.5) ** 0.5)), axis=1)
                        img_NBR = np.expand_dims(((NIR - SWIR_2) / (NIR + SWIR_2)), axis=1)
                        img_NBVI = np.expand_dims((NIR / Green), axis=1)
                        img_NDGI = np.expand_dims(((Green - Red) / (Green + Red)), axis=1)
                        img_NDSI = np.expand_dims(((SWIR_1 - NIR) / (SWIR_1 + NIR)), axis=1)
                        img_NDVI = np.expand_dims(((NIR - Red) / (NIR + Red)), axis=1)
                        img_NDWI = np.expand_dims(((Green - NIR) / (Green + NIR)), axis=1)
                        img_NG = np.expand_dims((Green / (NIR + Red + Green)), axis=1)
                        img_NLI = np.expand_dims(((NIR ** 2 - Red) / (NIR ** 2 + Red)), axis=1)
                        img_NMDI = np.expand_dims(((NIR - (SWIR_1 - SWIR_2)) / (NIR + (SWIR_1 - SWIR_2))), axis=1)
                        img_NR = np.expand_dims((Red / (NIR + Red + Green)), axis=1)
                        img_OSAVI = np.expand_dims(((NIR - Red) / (NIR + Red + 0.16)), axis=1)
                        img_PSRI = np.expand_dims(((Red - Green) / NIR), axis=1)
                        img_RDVI_1 = np.expand_dims(((NIR - Red) / ((NIR + Red) ** 0.5)), axis=1)
                        img_RVI = np.expand_dims((NIR / Red), axis=1)
                        img_SARVI = np.expand_dims(((2 - ((NIR - Red) / (NIR + Red))) * (NIR - (2 * Red - Blue)) / (NIR + (2 * Red - Blue) + 1 - ((NIR - Red) / (NIR + Red)))), axis=1)
                        img_SAVI_2 = np.expand_dims(((2 - ((NIR - Red) / (NIR + Red))) * (NIR - Red) / (NIR + Red + 1 - ((NIR - Red) / (NIR + Red)))), axis=1)
                        img_SBL = np.expand_dims((NIR - 2.4 * Red), axis=1)
                        img_SI = np.expand_dims(((((1 - Blue) * (1 - Green) * (1 - Red)) ** (1 / 3))), axis=1)
                        img_SIPI = np.expand_dims(((NIR - Blue) / (NIR + Red)), axis=1)
                        img_SLAVI = np.expand_dims((NIR / (Red + SWIR_2)), axis=1)
                        img_TGI = np.expand_dims(((0.1725 * (Red - Green) - 0.0925 * (Red - Blue)) / 2), axis=1)
                        img_VARI = np.expand_dims(((Green - Red) / (Green + Red - Blue)), axis=1)
                        img_VDVI = np.expand_dims(((2 * Green - Red - Blue) / ((2 * Green + Red + Blue))), axis=1)

                        # img_w_ind = np.concatenate([img_flat, img_AFVI1600, img_ARVI, img_ATSAVI, img_BI_1, img_BI_2, img_CSI, img_DVI, img_EVI, img_GARI_0, img_GB_NDVI,
                        #                 img_GSAVI, img_IPVI, img_Laterite, img_MIRBI, img_MNDWI, img_MSAVI, img_MTVI_2, img_NBR, img_NBVI, img_NDGI,
                        #                 img_NDSI, img_NDVI, img_NDWI, img_NG, img_NLI, img_NMDI, img_NR, img_OSAVI, img_PSRI, img_RDVI_1,
                        #                 img_RVI, img_SARVI, img_SAVI_2, img_SBL, img_SI, img_SIPI, img_SLAVI, img_TGI, img_VARI, img_VDVI,], axis=1)
                        
                        # # -----------------------------------------------------------------------------------------------------------
                        # # Paras003
                        # ###, Blue, Green, Red, SWIR_1, SWIR_2, AFVI1600, ARVI, BI_1, BI_2, GARI_0, GB_NDVI, GSAVI, Laterite, MNDWI, MTVI_2, NDGI, NG, NMDI, NR, PSRI, RDVI_1, SI, SIPI, VDVI, Aspect, DEM
                        # ###, 1, 2, 3, 4, 5, 6, 17, 18, 20, 21, 25, 26, 27, 29, 31, 33, 36, 40, 42, 43, 45, 46, 51, 52, 56, 58, 60
                        img_w_ind = (np.c_[
                                    Blue, Green, Red, NIR, SWIR_1, SWIR_2,
                                    img_AFVI1600, img_ARVI,  img_BI_1,     img_BI_2,  img_GARI_0,
                                    img_GB_NDVI,  img_GSAVI, img_Laterite, img_MNDWI, img_MTVI_2,
                                    img_NDGI,     img_NG,    img_NMDI,     img_NR,    img_PSRI,
                                    img_RDVI_1,   img_SI,    img_SIPI,     img_VDVI,
                                    Aspect, DEM
                        ])
        
                        # to_predict = pd.DataFrame(img_w_ind)
                        # (rows, cols) = to_predict.shape  # 查看数据大小，行列
                        # print('dataPre.shape',to_predict.shape)
                        # dataPre = (np.array(to_predict))
                    
                        # 对数据进行预处理
                        dataPre = pd.DataFrame(img_w_ind)
                        nanindex = dataPre.index[np.isnan(dataPre).T.any()]       # 统计出空值 nan 所在的行号
                        # ic(nanindex)
                        infindex = dataPre.index[np.isinf(dataPre).T.any()]       # 统计出空值 inf 所在的行号
                        # ic(infindex)
                        (rows, cols) = dataPre.shape  # 查看数据大小，行列
                        
                        dataPre = (np.array(dataPre))
                        dataPre[np.isnan(dataPre)] = 255
                        dataPre[np.isinf(dataPre)] = 255

                        img_preds = loaded_modelR.predict(dataPre)
                        img_preds = img_preds  

                        img_preds[(img_NDVI.flatten() < 0.25) | (img_NDVI.flatten() > 1) | (img_RVI.flatten() <= 2)] = 0
                        img_preds[(Red <= 0) | (Red >= 1) | (Green <= 0) | (Green >= 1) | (Blue <= 0) | (Blue >= 1)] = 0
                        
                        img_preds[nanindex] = 255
                        img_preds[infindex] = 255
                        img_preds[img_preds == 255] = 0  # 将255赋值为0


                        img_preds[img_preds < 0] = 0
                        img_preds[img_preds > 1] = 1

                        img_preds = img_preds * 100

                        img_preds = img_preds.astype(np.uint8)  # astype(np.uint8)

                        output = img_preds.reshape(*img_swp.shape[:-1])
                        # print('(*img_swp.shape[:-1])',(*img_swp.shape[:-1]))
                        output = output.reshape(*img_swp.shape[:-1])
                        # print('output.shape',output.shape)

                        dst.write(output.astype(rasterio.uint8), 1, window=window)   # uint8类型取值范围：0到255（通常用于RGB图像中）


[03:12:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
GEE_Colab_DC2_MER_TRA_40Vis-0Samp02_005Class2_DC2_MER_Pre_40Vis_005Class2_XGBR1.00_N100_Paras003-02_model.sav
Save Path：-----> /content/drive/Shareddrives/liuxb555@gmail.com/Landsat_05151015/Landsat_Median_TNR/L8_202005151015_Median_TNR_FFC
len(tiffList)  108
(len(tiffList)) 108 num_i 82
output_image:  /content/drive/Shareddrives/liuxb555@gmail.com/Landsat_05151015/Landsat_Median_TNR/L8_202005151015_Median_TNR_FFC/Pred_L8_202005151015_Median_TNR-0000052480-0000146944_GEE_Colab_DC2-DC2_XGBR1.00_N100_Paras003-02_FFC.tif
